<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/%D0%9F%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D1%8B_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA%D0%B8_28_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h3 id="Процесс-и-его-характеристики">Процесс и его характеристики<a class="anchor-link" href="#Процесс-и-его-характеристики"></a></h3><ul>
<li>Что такое процесс?</li>
<li>Какие процессы запущены в ОС?</li>
<li>Как запустить python процесс?</li>
<li>Что делает процесс во время исполнения?</li>
</ul>



<p>Характеристики процесса:</p>
<ul>
<li>Идентификатор процесса, PID</li>
<li>Объем оперативной памяти</li>
<li>Стек</li>
<li>Список открытых файлов</li>
<li>Ввод/вывод</li>
</ul>


In [1]:
# простой Python процесс

import time
import os

pid = os.getpid()

while True:
    print(pid, time.time())
    time.sleep(2)

64 1635424532.978097
64 1635424534.9913707
64 1635424536.9941223
64 1635424538.9964132
64 1635424540.998703
64 1635424543.001004
64 1635424545.0032961
64 1635424547.0055804


KeyboardInterrupt: ignored


<h3 id="Создание-процесса-на-Python">Создание процесса на Python<a class="anchor-link" href="#Создание-процесса-на-Python"></a></h3><ul>
<li>Как создать дочерний процесс?</li>
<li>Как работает системный вызов fork?</li>
<li>Модуль multiprocessing</li>
</ul>


In [2]:
# Создание процесса на Python

import time
import os

pid = os.fork()
if pid == 0:
    # дочерний процесс
    while True:
        print("child:", os.getpid())
        time.sleep(5)
else:
    # родительский процесс
    print("parent:", os.getpid())
    os.wait()

child: 113
parent: 64
child: 113
child: 113
child: 113
child: 113
child: 113


KeyboardInterrupt: ignored

KeyboardInterrupt: ignored

In [3]:
# Память родительского и дочернего процесса

import os

foo = "bar"

if os.fork() == 0:
    # дочерний процесс
    foo = "baz"
    print("child:", foo)
else:
    # родительский процесс
    print("parent:", foo)
    os.wait()

child: baz
parent: bar


KeyboardInterrupt: ignored

In [4]:
# Файлы в родительском и дочернем процессе

! echo example string1 >> data.txt
! echo example string2 >> data.txt

import os

f = open("data.txt")
foo = f.readline()

if os.fork() == 0:
    # дочерний процесс
    foo = f.readline()
    print("child:", foo)
else:
    # родительский процесс
    foo = f.readline()
    print("parent:", foo)

parent: example string2

child: example string2



In [5]:
# Создание процесса, модуль multiprocessing

from multiprocessing import Process

def f(name):
    print("hello", name)

p = Process(target=f, args=("Bob",))
p.start()
p.join()

hello Bob


In [6]:
# Создание процесса, модуль multiprocessing

from multiprocessing import Process

class PrintProcess(Process):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        print("hello", self.name)

p = PrintProcess("Mike")
p.start()
p.join()

hello Mike



<h3 id="Создание-потоков">Создание потоков<a class="anchor-link" href="#Создание-потоков"></a></h3><ul>
<li>Что такое поток</li>
<li>Создание потоков, модуль threading</li>
<li>Использование ThreadPoolExecutor</li>
</ul>



<h3 id="Создание-потоков">Создание потоков<a class="anchor-link" href="#Создание-потоков"></a></h3><ul>
<li>Поток напоминает процесс</li>
<li>У потока своя последовательность инструкций</li>
<li>Каждый поток имеет собственный стек</li>
<li>Все потоки выполняются в рамках процесса</li>
<li>Потоки разделяют память и ресурсы процесса</li>
<li>Управлением выполнением потоков занимается ОС</li>
<li>Потоки в Python имеют свои ограничения</li>
</ul>


In [7]:
# Создание потока

from threading import Thread

def f(name):
    print("hello", name)

th = Thread(target=f, args=("Bob",))
th.start()
th.join()

hello Bob


In [8]:
# Создание потока

from threading import Thread

class PrintThread(Thread):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        print("hello", self.name)

th = PrintThread("Mike")
th.start()
th.join()

hello Mike


In [9]:
# Пул потоков, concurrent.futures.Future

from concurrent.futures import ThreadPoolExecutor, as_completed

def f(a):
    return a * a

# .shutdown() in exit
with ThreadPoolExecutor(max_workers=3) as pool:
    results = [pool.submit(f, i) for i in range(10)]

    for future in as_completed(results):
        print(future.result())

1
0
4
9
16
25
36
49
64
81



<h3 id="Синхронизация-потоков">Синхронизация потоков<a class="anchor-link" href="#Синхронизация-потоков"></a></h3><ul>
<li>Очереди</li>
<li>Блокировки</li>
<li>Условные переменные</li>
</ul>


В многопоточной программе доступ к объектам иногда нужно синхронизировать.
Часто для синхронизации потоков используют блокировки.
Любые блокировки замедляют выполнение программы.

Лучше избегать использование блокировок 
и отдавать предпочтение обмену данными через очереди.

In [27]:
# Очереди, модуль queue
from queue import Queue
from threading import Thread

def worker(q, n):
    while True:
        item = q.get()
        if item is None:
            break
        print("process data:", n, item)

q = Queue(5)
th1 = Thread(target=worker, args=(q, 1))
th2 = Thread(target=worker, args=(q, 2))
th1.start(); th2.start()

for i in range(40):
    q.put(i)

q.put(None); q.put(None)
th1.join(); th2.join()

process data: 1 0
process data: 1 1
process data: 1 2
process data: 1 3
process data: 1 4
process data: 1 5
process data: 1 6
process data: 1 7
process data: 1 8
process data: 1 9
process data: 1 10
process data: 1 11
process data: 1 12
process data: 1 13
process data: 1 14
process data: 1 15
process data: 1 16
process data: 1 17
process data: 1 18
process data: 1 19
process data: 1 20
process data: 1 21
process data: 1 22
process data: 1 23
process data: 1 24
process data: 1 25
process data: 1 26
process data: 1 27
process data: 1 28
process data: 1 29
process data: 1 30
process data: 1 31
process data:process data: 2  33
process data: 2 34
process data: 2 35
1 process data: 2 36
process data: 232
process data: 1  37
process data: 2 3938



Создаем очередь с максимальным размером 5.  
Используем методы `put()` для того, чтобы поместить данные в очередь
и `get()` для того, чтобы забрать данные из очереди.

Использование очередей делает код выполняемой программы более простым.
И по возможности лучше разрабатывать код таким образом,
чтобы не было глобального разделяемого ресурса или состояния.

In [ ]:
# Синхронизация потоков, race condition

import threading

class Point(object):
    def __init__(self, x, y):
        self.set(x, y)

    def get(self):
        return (self.x, self.y)

    def set(self, x, y):
        self.x = x
        self.y = y

# use in threads
my_point = Point(10, 20)
my_point.set(15, 10)
my_point.get()

(15, 10)

In [ ]:
# Синхронизация потоков, блокировки

import threading

class Point(object):
    def __init__(self, x, y):
        self.mutex = threading.RLock()
        self.set(x, y)

    def get(self):
        with self.mutex:
            return (self.x, self.y)

    def set(self, x, y):
        with self.mutex:
            self.x = x
            self.y = y

# use in threads
my_point = Point(10, 20)
my_point.set(15, 10)
my_point.get()

(15, 10)

Этот код гарантирует, что если объект класса Point будет использоваться в разных потоках, то изменение x и y будет всегда атомарным.

Работает все это так: при вызове метода берем блокировку через with self._mutex.  
Весь код внутри with блока будет выполнятся только в одном потоке.

Другими словами, если два разных потока вызовут .get то пока первый поток не выйдет из блока 
второй будет его ждать - и только потом продолжит выполнение.

Зачем это все нужно? Координаты нужно менять одновременно - ведь точка это атомарный объект.
Если позволить одному потоку поменять x, а другой в это же время поправит, y
логика алгоритма может сломаться.

In [ ]:
# Синхронизация потоков, блокировки

import threading


a = threading.RLock()
b = threading.RLock()

def foo():
    try:
        a.acquire()
        b.acquire()
    finally:
        a.release()
        b.release()

In [ ]:
# Синхронизация потоков, условные переменные

class Queue(object):
    def __init__(self, size=5):
        self._size = size
        self._queue = []
        self._mutex = threading.RLock()
        self._empty = threading.Condition(self._mutex)
        self._full = threading.Condition(self._mutex)
    
    def put(self, val):
        with self._full:
            while len(self._queue) >= self._size:
                self._full.wait()
            
            self._queue.append(val)
            self._empty.notify()

    def get(self):
        with self._empty:
            while len(self._queue) == 0:
                self._empty.wait()
            
            ret = self._queue.pop(0)
            self._full.notify()
            return ret


<h3 id="Глобальная-блокировка-интерпретатора,-GIL">Глобальная блокировка интерпретатора, GIL<a class="anchor-link" href="#Глобальная-блокировка-интерпретатора,-GIL"></a></h3><ul>
<li>Что такое Global Interpreter Lock?</li>
<li>Зачем нужен GIL?</li>
<li>GIL и системные вызовы</li>
</ul>


GIL - это достаточно сложная тема в Python.
Для более глубокого понимания того, как работают потоки,
нужно иметь общее представление, зачем нужен GIL и как он устроен.

GIL защищает память интерпретатора от повреждений и делает операции атомарными.

Поток, владеющий GIL, не отдает его, пока об этом не попросят.
Потоки засыпают на 5 мс. для ожидания GIL.
Сам GIL устроен как обычная нерекурсивная блокировка. Эта же структура лежит в основе threading.Lock. 

Когда Python делает системный вызов или вызов из внешней библиотеки, он отключает механизм GIL.
После того, как функция вернет управление, снова включает его.

Т.е. потоки при своем выполнении так или иначе вынуждены получать GIL.
Именно поэтому многопоточные программы, требующие больших вычислений,
могут выполняться медленней, чем однопоточные.

In [ ]:
# cpu bound programm

from threading import Thread
import time

def count(n):
    while n > 0:
        n -= 1

# series run
t0 = time.time()
count(100_000_000)
count(100_000_000)
print(time.time() - t0)

# parallel run
t0 = time.time()
th1 = Thread(target=count, args=(100_000_000,))
th2 = Thread(target=count, args=(100_000_000,))

th1.start(); th2.start()
th1.join(); th2.join()
print(time.time() - t0)

28.54827880859375
27.99934434890747


In [ ]:
# как выполняется поток?

a      r      a            r              a          r    a
  run  |------|    run     |--------------|   run    |----| run
------>|  IO  |----------->|      IO      |--------->| IO |----->
       |------|            |--------------|          |----|
a      r      a            r              a          r    a

a - acquire GIL
r - release GIL